In [1]:
import numpy as np
import pandas as pd
import string
import re
import warnings
import os
import snscrape
import snscrape.modules.twitter as sntwitter
import datetime
import csv
import mysql.connector as c
    
warnings.filterwarnings(action='ignore')

In [2]:
mydb = c.connect(host = "localhost", user = "root", password = "0000", database = "TWITTER",auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

init="DROP TABLES IF EXISTS TWITTER"
mycursor.execute(init)

sql="CREATE TABLE TWITTER(I INT NOT NULL AUTO_INCREMENT,URL VARCHAR(100),USERNAME VARCHAR(20),DATE DATE,TIME TIME,TEXT VARCHAR(500),PRIMARY KEY(I));"
mycursor.execute(sql)

ins="INSERT INTO TWITTER(I,URL,USERNAME,DATE,TIME,TEXT) VALUES(NULL,%s,%s,%s,%s,%s)"

In [3]:
def tweet_list():
    # Set maximum tweets to pull
    maxTweets = 3000
    # Set what keywords you want your twitter scraper to pull
    keyword = '한강 의대생'
    #Open/create a file to append data to
    csvFile = open('tweets_result.csv', 'w', newline='', encoding='utf-8-sig')
    #Use csv writer
    csvWriter = csv.writer(csvFile)
    csvWriter.writerow(['url','username','date','time','tweet'])

    # Write tweets into the csv file
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword + ' lang:ko since:2021-04-25 until:2021-09-30 include:nativeretweets').get_items()):
            if i > maxTweets :
                break  
            date=int(datetime.datetime.strptime(str(tweet.date)[:-6],"%Y-%m-%d %H:%M:%S").timestamp())+32400
            #print(date)
            date=datetime.datetime.fromtimestamp(int(date)).strftime('%Y-%m-%d %H:%M:%S')
            #print(date)
            csvWriter.writerow([tweet.url,'@'+tweet.username, str(date).split()[0],str(date).split()[1], tweet.content])
            URL=tweet.url
            #print(URL)
            USERNAME= '@'+tweet.username
            DATE=str(tweet.date).split()[0]
            TIME=str(tweet.date).split()[1]
            TEXT=tweet.content
            tup=(URL,USERNAME,DATE,TIME,TEXT)
            mycursor.execute(ins,tup)
    csvFile.close()
    df=pd.read_csv('tweets_result.csv')
    df.head(50)

    mydb.commit()
    mydb.close()

In [4]:
tweet_list()